In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StringType, LongType

In [2]:
spark = SparkSession.builder\
    .master('local')\
    .appName('homework_lesson_14')\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
df_aisles = spark.read.option("header", True).option("inferSchema", True).csv("/bronze/dshop/aisles")
df_clients = spark.read.option("header", True).option("inferSchema", True).csv("/bronze/dshop/clients")
df_departments = spark.read.option("header", True).option("inferSchema", True).csv("/bronze/dshop/departments")
df_orders = spark.read.option("header", True).option("inferSchema", True).csv("/bronze/dshop/orders")
df_products = spark.read.option("header", True).option("inferSchema", True).csv("/bronze/dshop/products")

In [5]:
df_aisles.show()

+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|       1|prepared soups sa...|
|       2|   specialty cheeses|
|       3| energy granola bars|
|       4|       instant foods|
|       5|marinades meat pr...|
|       6|               other|
|       7|       packaged meat|
|       8|     bakery desserts|
|       9|         pasta sauce|
|      10|    kitchen supplies|
|      11|    cold flu allergy|
|      12|         fresh pasta|
|      13|      prepared meals|
|      14|tofu meat alterna...|
|      15|    packaged seafood|
|      16|         fresh herbs|
|      17|  baking ingredients|
|      18|bulk dried fruits...|
|      19|       oils vinegars|
|      20|        oral hygiene|
+--------+--------------------+
only showing top 20 rows



In [6]:
df_clients.show()

+---------+---------------+----------------+
|client_id|       fullname|location_area_id|
+---------+---------------+----------------+
|        1| Anthony Reilly|               2|
|        2|Christina Boyle|               2|
|        3|  Andrew Walker|               1|
|        4|Emily Rodriguez|               3|
|        5|    Glen Travis|               2|
|        6|     Adam Mayer|               8|
|        7|  Lydia Griffin|               6|
|        8|     Marcus Cox|               3|
|        9|   Sean Johnson|               6|
|       10|    Joel Mullen|               1|
|       11|   Laurie Brown|               3|
|       12|  Kevin Johnson|               7|
|       13|    Paula Lopez|               3|
|       14|Michelle Hodges|               1|
|       15|   Jerome Russo|               6|
|       16|Jonathon Porter|               2|
|       17| Andrew Jackson|               2|
|       18|   Emma Jackson|               7|
|       19|     Pam Wilson|               4|
|       20

In [7]:
df_departments.show()

+-------------+---------------+
|department_id|     department|
+-------------+---------------+
|            1|         frozen|
|            2|          other|
|            3|         bakery|
|            4|        produce|
|            5|        alcohol|
|            6|  international|
|            7|      beverages|
|            8|           pets|
|            9|dry goods pasta|
|           10|           bulk|
|           11|  personal care|
|           12|   meat seafood|
|           13|         pantry|
|           14|      breakfast|
|           15|   canned goods|
|           16|     dairy eggs|
|           17|      household|
|           18|         babies|
|           19|         snacks|
|           20|           deli|
+-------------+---------------+
only showing top 20 rows



In [8]:
df_orders.show()

+--------+----------+---------+--------+--------+----------+
|order_id|product_id|client_id|store_id|quantity|order_date|
+--------+----------+---------+--------+--------+----------+
|       1|     13049|      835|      26|       6|2021-01-01|
|       1|     40200|      835|      26|       3|2021-01-01|
|       1|     31576|      835|      26|       1|2021-01-01|
|       1|     39380|      835|      26|       4|2021-01-01|
|       1|     14430|      835|      26|       3|2021-01-01|
|       1|     24859|      835|      26|       3|2021-01-01|
|       1|     12849|      835|      26|       3|2021-01-01|
|       1|      1034|      835|      26|       2|2021-01-01|
|       1|     37554|      835|      26|       5|2021-01-01|
|       1|     16264|      835|      26|       5|2021-01-01|
|       1|     14280|      835|      26|       6|2021-01-01|
|       1|     13389|      835|      26|       2|2021-01-01|
|       1|     25695|      835|      26|       4|2021-01-01|
|       1|     23323|   

In [9]:
df_products.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
|         6|        Dry Nose Oil|      11|           11|
|         7|Pure Coconut Wate...|      98|            7|
|         8|Cut Russet Potato...|     116|            1|
|         9|Light Strawberry ...|     120|           16|
|        10|Sparkling Orange ...|     115|            7|
|        11|   Peach Mango Juice|      31|            7|
|        12|Chocolate Fudge L...|     119|            1|
|        13|   Saline Nasal Mist|      11|           11|
|        14|Fresh Scent Dishw...|      74|           17|
|        15|Overnight Diapers..

In [10]:
df = df_orders

In [11]:
df = df.groupby(F.col("order_id"), F.col("product_id"), F.col("client_id"), F.col("store_id"), F.col("order_date"))\
        .agg(F.sum(F.col('quantity')))\
        .select(F.col("order_id"), F.col("product_id"), F.col("client_id"), F.col("store_id"), F.col("order_date"), F.col('sum(quantity)').alias('quantity'))

In [12]:
df.show()

+--------+----------+---------+--------+----------+--------+
|order_id|product_id|client_id|store_id|order_date|quantity|
+--------+----------+---------+--------+----------+--------+
|      15|      1541|      128|      19|2021-01-01|       6|
|      18|     43872|     1079|      10|2021-01-01|       4|
|      27|     21609|     1322|      11|2021-01-01|       2|
|      31|     12692|      207|      21|2021-01-01|       6|
|      36|     33106|      196|      17|2021-01-01|       2|
|      43|     14608|     1292|      21|2021-01-01|       1|
|      55|     13853|      219|       6|2021-01-01|       2|
|      70|     29081|      195|       1|2021-01-01|       6|
|      75|     42484|      371|      31|2021-01-01|       3|
|      93|     18933|     1200|      31|2021-01-01|       1|
|      99|     32150|      201|       7|2021-01-01|       4|
|     114|     43106|      494|       9|2021-01-01|       1|
|     133|      7141|      317|       1|2021-01-01|       2|
|     158|     22591|   

In [13]:
df.where((df.order_id == 4173) & (df.product_id == 9371)).sort('product_id').show()

+--------+----------+---------+--------+----------+--------+
|order_id|product_id|client_id|store_id|order_date|quantity|
+--------+----------+---------+--------+----------+--------+
|    4173|      9371|       38|       3|2021-01-17|      11|
+--------+----------+---------+--------+----------+--------+



In [14]:
df_silver_aisles = spark.read.parquet("/silver/dshop/aisles")
df_silver_clients = spark.read.parquet("/silver/dshop/clients")
df_silver_departments = spark.read.parquet("/silver/dshop/departments")
df_silver_orders = spark.read.parquet("/silver/dshop/orders")
df_silver_products = spark.read.parquet("/silver/dshop/products")

In [15]:
df_silver_aisles.show()

+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|       1|prepared soups sa...|
|       2|   specialty cheeses|
|       3| energy granola bars|
|       4|       instant foods|
|       5|marinades meat pr...|
|       6|               other|
|       7|       packaged meat|
|       8|     bakery desserts|
|       9|         pasta sauce|
|      10|    kitchen supplies|
|      11|    cold flu allergy|
|      12|         fresh pasta|
|      13|      prepared meals|
|      14|tofu meat alterna...|
|      15|    packaged seafood|
|      16|         fresh herbs|
|      17|  baking ingredients|
|      18|bulk dried fruits...|
|      19|       oils vinegars|
|      20|        oral hygiene|
+--------+--------------------+
only showing top 20 rows



In [16]:
df_silver_clients.show()

+---------+---------------+----------------+
|client_id|       fullname|location_area_id|
+---------+---------------+----------------+
|        1| Anthony Reilly|               2|
|        2|Christina Boyle|               2|
|        3|  Andrew Walker|               1|
|        4|Emily Rodriguez|               3|
|        5|    Glen Travis|               2|
|        6|     Adam Mayer|               8|
|        7|  Lydia Griffin|               6|
|        8|     Marcus Cox|               3|
|        9|   Sean Johnson|               6|
|       10|    Joel Mullen|               1|
|       11|   Laurie Brown|               3|
|       12|  Kevin Johnson|               7|
|       13|    Paula Lopez|               3|
|       14|Michelle Hodges|               1|
|       15|   Jerome Russo|               6|
|       16|Jonathon Porter|               2|
|       17| Andrew Jackson|               2|
|       18|   Emma Jackson|               7|
|       19|     Pam Wilson|               4|
|       20

In [17]:
df_silver_departments.show()

+-------------+---------------+
|department_id|     department|
+-------------+---------------+
|            1|         frozen|
|            2|          other|
|            3|         bakery|
|            4|        produce|
|            5|        alcohol|
|            6|  international|
|            7|      beverages|
|            8|           pets|
|            9|dry goods pasta|
|           10|           bulk|
|           11|  personal care|
|           12|   meat seafood|
|           13|         pantry|
|           14|      breakfast|
|           15|   canned goods|
|           16|     dairy eggs|
|           17|      household|
|           18|         babies|
|           19|         snacks|
|           20|           deli|
+-------------+---------------+
only showing top 20 rows



In [18]:
df_silver_orders.show()

+--------+----------+---------+--------+----------+--------+
|order_id|product_id|client_id|store_id|order_date|quantity|
+--------+----------+---------+--------+----------+--------+
|       4|     36483|     1032|      15|2021-01-01|       4|
|      11|     11241|     1189|       8|2021-01-01|       6|
|      12|     28491|      977|      19|2021-01-01|       3|
|      15|     46958|      128|      19|2021-01-01|       5|
|      17|     27577|      569|       1|2021-01-01|       4|
|      40|     30301|      200|      16|2021-01-01|       1|
|      61|     36424|      823|       5|2021-01-01|       6|
|      67|     35372|      782|      10|2021-01-01|       3|
|      81|     41303|     1350|      15|2021-01-01|       5|
|      81|      7584|     1350|      15|2021-01-01|       6|
|      82|     23640|     1239|      24|2021-01-01|       6|
|      86|     31380|      531|      30|2021-01-01|       5|
|      89|       100|       46|      32|2021-01-01|       2|
|     119|      6457|   

In [19]:
df_silver_products.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
|         6|        Dry Nose Oil|      11|           11|
|         7|Pure Coconut Wate...|      98|            7|
|         8|Cut Russet Potato...|     116|            1|
|         9|Light Strawberry ...|     120|           16|
|        10|Sparkling Orange ...|     115|            7|
|        11|   Peach Mango Juice|      31|            7|
|        12|Chocolate Fudge L...|     119|            1|
|        13|   Saline Nasal Mist|      11|           11|
|        14|Fresh Scent Dishw...|      74|           17|
|        15|Overnight Diapers..

In [20]:
df_silver_products.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
|         6|        Dry Nose Oil|      11|           11|
|         7|Pure Coconut Wate...|      98|            7|
|         8|Cut Russet Potato...|     116|            1|
|         9|Light Strawberry ...|     120|           16|
|        10|Sparkling Orange ...|     115|            7|
|        11|   Peach Mango Juice|      31|            7|
|        12|Chocolate Fudge L...|     119|            1|
|        13|   Saline Nasal Mist|      11|           11|
|        14|Fresh Scent Dishw...|      74|           17|
|        15|Overnight Diapers..

In [21]:
df_silver_orders.where((df_silver_orders.order_id == 4173) & (df_silver_orders.product_id == 9371)).show()

+--------+----------+---------+--------+----------+--------+
|order_id|product_id|client_id|store_id|order_date|quantity|
+--------+----------+---------+--------+----------+--------+
|    4173|      9371|       38|       3|2021-01-17|      11|
+--------+----------+---------+--------+----------+--------+



In [22]:
json_schema = StructType()\
    .add('date', StringType())\
    .add('product_id', LongType())

In [23]:
df_out_of_stock_app = spark.read.schema(json_schema).json("/bronze/out_of_stock_app/*/*/*")

In [24]:
df_out_of_stock_app.show()

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-04-05|     33204|
|2021-04-05|     26105|
|2021-04-05|     30030|
|2021-04-05|      2230|
|2021-04-05|     22527|
|2021-04-05|     29338|
|2021-04-05|     35862|
|2021-04-05|     32281|
|2021-04-05|     40513|
|2021-04-05|      3444|
|2021-04-05|     32447|
|2021-04-05|      6640|
|2021-04-05|     17967|
|2021-04-05|     25008|
|2021-04-05|     20359|
|2021-04-05|     37231|
|2021-04-05|     15073|
|2021-04-05|      1299|
|2021-04-05|     40349|
|2021-04-05|      6212|
+----------+----------+
only showing top 20 rows



In [25]:
df_out_of_stock_app.groupby('date').count().sort('date').show()

+----------+-----+
|      date|count|
+----------+-----+
|2021-04-01| 2084|
|2021-04-02| 2283|
|2021-04-03| 1660|
|2021-04-04| 2232|
|2021-04-05| 2741|
+----------+-----+



In [27]:
df_silver_out_of_stock_app = spark.read.parquet("/silver/out_of_stock_app/")

In [28]:
df_silver_out_of_stock_app.show()

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-04-05|      2230|
|2021-04-05|     41222|
|2021-04-05|        27|
|2021-04-05|     10408|
|2021-04-05|     35104|
|2021-04-05|     16333|
|2021-04-05|     23488|
|2021-04-05|     14724|
|2021-04-05|     27289|
|2021-04-05|     35627|
|2021-04-05|      5447|
|2021-04-05|     33880|
|2021-04-05|     26304|
|2021-04-05|     48976|
|2021-04-05|     12116|
|2021-04-05|     40804|
|2021-04-05|     16157|
|2021-04-05|     35437|
|2021-04-05|     24952|
|2021-04-05|     20849|
+----------+----------+
only showing top 20 rows



In [29]:
df_silver_out_of_stock_app.groupby('date').count().sort('date').show()

+----------+-----+
|      date|count|
+----------+-----+
|2021-04-01| 2036|
|2021-04-02| 2229|
|2021-04-03| 1633|
|2021-04-04| 2177|
|2021-04-05| 2680|
+----------+-----+

